In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [17]:
from sklearn import tree, neighbors, svm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [5]:
tv_data = pd.read_csv("aggregated_data.csv", usecols = ["week", "day", "hour", "sub_genre_id", "channel_id", "users", "program_id"])

In [6]:
tv_data["day_since"] = ((tv_data["week"] -1 )* 7) + tv_data["day"]

In [7]:
tv_data["new"] = int(0)

In [8]:
tv_data.head()

,week,day,hour,sub_genre_id,channel_id,program_id,users,day_since,new
0,1,1,1,1,181,366653,5,1,0
1,1,1,1,3,1,367167,20,1,0
2,1,1,1,3,4,68785,34,1,0
3,1,1,1,5,38,11568,5,1,0
4,1,1,1,9,6,177908,90,1,0


In [10]:
unique_programs_filter = tv_data.drop_duplicates(subset =["program_id"], keep = "first", inplace = False)
unique_programs_filter["new"] = int(1)
tv_data["new"].update(unique_programs_filter["new"].astype(int))

/home/yogesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
#tv_data[tv_data["new"] == 1]

In [12]:
train_data = tv_data[(tv_data["week"] < 13) & (tv_data["new"] == 1) ]

test_data = tv_data[(tv_data["week"] == 13)  & (tv_data["new"] == 1) & (tv_data["channel_id"].isin(train_data["channel_id"].unique()))  ]

In [13]:
print (test_data.shape)
print (train_data.shape)

(582, 9)
(12540, 9)


In [14]:
Y_train = train_data["users"]
X_train = train_data[["day", "hour", "channel_id", "sub_genre_id"]]

In [15]:
Y_test = test_data["users"]
X_test = test_data[["day", "hour", "channel_id", "sub_genre_id"]]


In [18]:
reg = GradientBoostingRegressor(n_estimators=1300, learning_rate=0.1, max_depth=5, subsample=1.0,max_features =1, random_state=0, loss='ls')

In [19]:
reg.fit(X_train.values, Y_train.values)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=1,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1300, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [20]:
print (mean_squared_error(reg.predict(X_train.values),Y_train.values))

1082.1757531099827


In [21]:
print (mean_squared_error(reg.predict(X_test.values),Y_test.values))

4118.853058377224


In [181]:
from joblib import dump, load
dump(reg, 'filename.joblib') 

['filename.joblib']

In [23]:
test_sample = test_data[(test_data["hour"] == 18) ]

In [24]:
test_sample["predicted"] = reg.predict(test_sample[[  "day", "hour", "channel_id", "sub_genre_id"]])

/home/yogesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
test_sample.sort_values(by=['users'], ascending=False)[:10]

,week,day,hour,sub_genre_id,channel_id,program_id,users,day_since,new,predicted
213986,13,5,18,16,5,378687,259,89,1,181.287332
204640,13,1,18,42,5,32572,183,85,1,178.255576
218895,13,7,18,3,5,42309,182,91,1,214.783649
204591,13,1,18,28,6,377386,177,85,1,164.710667
213987,13,5,18,16,20,378327,71,89,1,9.606925
218929,13,7,18,17,97,378123,60,91,1,61.630054
204544,13,1,18,3,173,345903,53,85,1,47.424463
218928,13,7,18,17,94,70896,36,91,1,25.832415
216404,13,6,18,9,134,378884,32,90,1,7.721009
213988,13,5,18,16,42,377956,19,89,1,-1.414314
